In [7]:
#Algoritmo para detectar se serão vendidos carros ou não. 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.pyplot import plot, show, draw, figure, cm
from mpl_toolkits.mplot3d import Axes3D

#Fonte de dados
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"

#Leitura dos dados do csv
dados = pd.read_csv(uri)

#Mapeamento de colunas
mapa = {
    "mileage_per_year" : "milhas_por_ano",
    "model_year" : "ano_do_modelo",
    "price" : "preco",
    "sold" : "vendido"
}

a_trocar = { 
    'no' : 0,
    'yes' : 1
}

#Formata o cabeçalho dos dados de acordo com o mapeamento
dados = dados.rename(columns = mapa)
dados.vendido = dados.vendido.map(a_trocar)

dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


In [8]:
from datetime import datetime

#Inserindo coluna que informa a idade do veículo
ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo 
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,21
1,1,7843,1998,40557.96,1,23
2,2,7109,2006,89627.50,0,15
3,3,26823,2015,95276.14,0,6
4,4,7935,2014,117384.68,1,7


In [9]:
#Inserindo coluna que mostra a informação de milhas_por_ano em quilômetros.
dados['km_por_ano'] = dados.milhas_por_ano * 1.60934

dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,21,35085.22134
1,1,7843,1998,40557.96,1,23,12622.05362
2,2,7109,2006,89627.50,0,15,11440.79806
3,3,26823,2015,95276.14,0,6,43167.32682
4,4,7935,2014,117384.68,1,7,12770.11290


In [10]:
#Remover colunas que não vou utilizar
dados = dados.drop(columns = ["Unnamed: 0", "milhas_por_ano", "ano_do_modelo"])

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,21,35085.22134
1,40557.96,1,23,12622.05362
2,89627.50,0,15,11440.79806
3,95276.14,0,6,43167.32682
4,117384.68,1,7,12770.11290


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#Separando os dados de treinamento e teste 
x = dados [["preco", "idade_do_modelo", "km_por_ano"]]
y = dados ["vendido"]

#Determinando o SEED
SEED = 5
np.random.seed(SEED)

raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)
print("Treinamos com %d elementos e testamos como %d elementos" % (len(raw_treino_x), len(raw_teste_x)))

Treinamos com 7500 elementos e testamos como 2500 elementos


In [14]:
#Criando o modelo, treinando e fazendo as previsões.

#Lembre-se que o SVC não deve ser usado com dados em várias escalas. Precisamos antes reescalar
#os dados. 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

#modelo = LinearSVC()
modelo = SVC()

modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 77.48%


In [15]:
#criando um baseline para verificar o quão bom foi a acurácia obtida anteriormente. 
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
previsoes = dummy_stratified.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

#Ou também podemos fazer:
acuracia = dummy_stratified.score(teste_x, teste_y)

A acurácia foi 58.00%
